# Hugging Face 初體驗

Hugging Face 是一個專注於機器學習和自然語言處理的開源平台。它提供了大量的模型、數據集和工具，支持多種任務如文本生成、圖像分類和語音識別。用戶可以輕鬆地搜索、下載和使用這些資源來開發和部署AI應用。這個平台對研究人員和開發者來說非常有用，因為它促進了技術共享和協作。

平台提供各類開源模型及資料集，也針對不同性太的任務進行分類，便於查找使用．

![Hugging Face](https://raw.githubusercontent.com/jonascheng/learning-transformer-with-huggingface/refs/heads/main/assets/images/huggingface.png)

我們來體驗幾個簡單的任務

## [Token Classification](https://huggingface.co/tasks/token-classification)

Token classification 是自然語言處理中的一種任務，旨在對文本中的每個 token（通常是單詞或子詞）進行分類。這種技術可以用來解決多種問題，以下是幾個例子：

* 命名實體識別（NER）：識別文本中的人名、地名、組織名等實體。例如，從句子「John is working in New York」中識別出「John」是人名，「New York」是地名。
* 詞性標註（POS tagging）：標註每個單詞的詞性，如名詞、動詞、形容詞等。例如，從句子「The quick brown fox jumps over the lazy dog」中標註出「quick」是形容詞，「jumps」是動詞。


In [ ]:
# transformers not support NumPy 2.0 yet
%pip install -q numpy==1.26.4 torch transformers

In [ ]:
from transformers import pipeline

# 命名實體識別（NER）, 讓 pipeline 自動選擇模型
ner_classifier = pipeline(task="ner")
ner_classifier("John is working in New York")

In [ ]:
# 詞性標註（POS tagging）, 指定模型
model = "vblagoje/bert-english-uncased-finetuned-pos"
pos_classifier = pipeline(
  task="token-classification",
  model=model,)
pos_classifier("The quick brown fox jumps over the lazy dog")